In [ ]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [ ]:
from flask import Response, request
from flask.globals import request_ctx
from seismic_foundation.foundation.ServiceProvider import ServiceProvider

from unittest.mock import MagicMock
from seismic_foundation.config.ConfigProvider import ConfigProvider
from services.ssrs_client import SSRSClient
from tests.test_constants import REPORT_DESCRIPTIONS, REPORT_METADATA
import seismic_auth as auth

from seismic_llm.langchain_adapters.chatLLMG import ChatLLMG

os.environ["CONFIG_PATH"] = "../config"
ServiceProvider.initialize()
config = ServiceProvider.get_service(ConfigProvider)

# hack while we wait for python-foundation to be fixed
config.matrix_server = "https://matrix-qa.seismic-dev.com"
config.matrix_certificate_password = config.matrix_certificate_password_hack
config.certificate_path = "../config/qa-matrix-cert.pfx"
auth.init(lambda: request, lambda: request_ctx, Response, "all")


In [ ]:
import pprint
from seismic_foundation.foundation.MatrixClient import MatrixClient
from seismic_foundation.foundation.AuthTokenClient import AuthTokenClient

ssrs_client = SSRSClient(
    ServiceProvider.get_service(MatrixClient),
    ServiceProvider.get_service(AuthTokenClient),
    config
)
# ssrs_client.get_all_reports("qadisc")
matrix = ServiceProvider.get_service(MatrixClient)
pprint.pp(ssrs_client.matrix.__dict__)

In [ ]:
REACT_SYSTEM_TEMPLATE = """
You are a helpful assistant that answers user questions by generating structured report metadata. 
You have access to the following tools:

{tools}

Your goal is to:
- Understand the user's question.
- Identify the most relevant reports (2-5) using `describe_available_reports`.
- Retrieve the reports' metadata using `fetch_metadata_by_report_id`.
- Filter down to the two most likely reports based on the user's question.
- Construct structured reports request using the available fields, filters, and sorting options.
- Execute any reports using `execute_report_request`.
- Analyze the results to determine if the generated reports meets the user's needs.

Provide your final answer in the following structured format:
Thoughts:
Reports:
  - report_name: <string>
    id: <UUID>
    fields:
      - <field 1>
      - <field 2>
    additional_filters:
      - filter_name: <string>
        operation: <operation type>
        filter_values:
          - <value 1>
          - <value 2>
    sort_by:
        field_name: <string>
        sort_order: ASC | DESC


Important rules:
- You must use at all tools before giving a Final Answer.
- Do not make up Observations. Wait for the actual result of the tool call.
- Use the tools to find the correct report, fields, filters, and sorting options.


Begin!

Question: {input}
Thought:{agent_scratchpad}
"""

In [ ]:
from models.agentic_react_model import AgenticReactModel

agentic_react_model = AgenticReactModel(
    config=config,
    ssrs_client=ssrs_client,
    tenant="qadisc",  # Use qadisc as the default tenant for testing
    llm_client=None
)


In [ ]:
from IPython.display import Image, display

display(Image(agentic_react_model.chain.get_graph().draw_mermaid_png()))


In [ ]:
agentic_react_model.answer_question("Who were top 10 publishers of livesends in April?")